### Test using Pydantic datamodels

In [1]:
from pydantic import BaseModel, Field
from typing import Optional
from typing import List, Union, Dict
import numpy as np
import datetime
from ngsildclient import Entity, Client

## Serialize directly to NGSI-LD

In [2]:
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
import yaml

class Property(BaseModel):
    type: str = "Property"
    value: Union[str, int, float, bool, Dict, object]

class AnimalDay(BaseModel):
    #type: str = "Property"
    observedAt : str
    methaneyield: float
    net_energy_requirement: float

class Day(BaseModel):
    type: str = "Day"
    id: str
    obs: Property
    pred: Property

obs = AnimalDay(observedAt=start.isoformat() + "Z", methaneyield= np.random.randint(30, 60),
                 net_energy_requirement= np.random.randint(30, 60))
pred = AnimalDay(observedAt=start.isoformat() + "Z", methaneyield= np.random.randint(30, 60),
                 net_energy_requirement= np.random.randint(30, 60))
#obs.__dict__
d = Day(id = Entity._build_fully_qualified_id("Day", "Cow005"), obs = Property(value=obs), pred = Property(value=pred))
e = Entity.from_json(d.json())
e.pprint()


{
  "type": "Day",
  "id": "urn:ngsi-ld:Day:Cow005",
  "obs": {
    "type": "Property",
    "value": {
      "observedAt": "2023-01-01T00:00:00Z",
      "methaneyield": 35.0,
      "net_energy_requirement": 38.0
    }
  },
  "pred": {
    "type": "Property",
    "value": {
      "observedAt": "2023-01-01T00:00:00Z",
      "methaneyield": 51.0,
      "net_energy_requirement": 45.0
    }
  },
  "@context": [
    "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"
  ]
}

In [13]:
import requests
ctx = "https://uri.etsi.org/ngsi-ld/default-context/"
headers = {
    "Content-Type": "application/ld+json"
}
r = requests.post("http://localhost:1026/ngsi-ld/v1/entities", 
                  headers=headers,
                  data=e.to_json())
print(r.status_code)
print(r.text)

201



In [6]:
client = Client(port=1026)
client.create(e)

Connected to Context Broker at localhost:1026 | tenant=N/A | vendor=Orion-LD | version=1.1.2

True

In [7]:
e1 = client.get(e, ctx= "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld")
d1 = Day(**e1.to_dict())
d1.obs.value

{'observedAt': '2023-01-01T00:00:00Z',
 'methaneyield': 37,
 'net_energy_requirement': 49}

### Simpler model -> custom serializer

In [8]:
class AnimalDay(BaseModel):
    methaneyield: float
    net_energy_requirement: float

class Day(BaseModel):
    type: str = "Day"
    id: str
    date: str
    observedAt: datetime.datetime
    obs: AnimalDay
    pred: AnimalDay
    
obs = AnimalDay(methaneyield=210, net_energy_requirement=600)
pred = AnimalDay(methaneyield=210, net_energy_requirement=600)

d = Day(id=Entity._build_fully_qualified_id("Day", "Cow007"),
    observedAt=start,
    obs=obs, pred=pred,
    date= start.date().isoformat())
e = Entity(d.type, d.id)
e.prop("date", d.date)
data = d.dict()
for k,v in data.items():
    if k in ["type", "id", "date"]:
        continue
    elif k != "observedAt":
        e.prop(k, v, observedat=d.observedAt)

e.pprint()

{
  "id": "urn:ngsi-ld:Day:Cow007",
  "type": "Day",
  "@context": [
    "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"
  ],
  "date": {
    "type": "Property",
    "value": "2023-01-01"
  },
  "obs": {
    "type": "Property",
    "value": {
      "methaneyield": 210.0,
      "net_energy_requirement": 600.0
    },
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "pred": {
    "type": "Property",
    "value": {
      "methaneyield": 210.0,
      "net_energy_requirement": 600.0
    },
    "observedAt": "2023-01-01T00:00:00Z"
  }
}

In [9]:
#Orion-ld should support keyvalue format -> TODO try to post directly without libary
#d.dict()

{'type': 'Day',
 'id': 'urn:ngsi-ld:Day:Cow007',
 'date': '2023-01-01',
 'observedAt': datetime.datetime(2023, 1, 1, 0, 0),
 'obs': {'methaneyield': 210.0, 'net_energy_requirement': 600.0},
 'pred': {'methaneyield': 210.0, 'net_energy_requirement': 600.0}}

In [10]:
client.update(e)

True

In [11]:
# Convert back to class, not probably very general method
# Might be better to iterate trough a dict instead
import ngsildclient
out = {}
for k,v in e.root.items():
    if type(v) is ngsildclient.model.attr.prop.AttrPropValue:
        out[k] = v["value"]
        if "observedAt" in v:
            out["observedAt"] = v["observedAt"]
    else:
        out[k] = v
Day(**out)
    

Day(type='Day', id='urn:ngsi-ld:Day:Cow007', date='2023-01-01', observedAt=datetime.datetime(2023, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), obs=AnimalDay(methaneyield=210.0, net_energy_requirement=600.0), pred=AnimalDay(methaneyield=210.0, net_energy_requirement=600.0))

# GPt4 code to compact dict

In [13]:
def compact_ngsi_ld_document(document: dict):
    compacted_document = document.copy()
    
    for key, value in document.items():
        if isinstance(value, dict) and "type" in value and value["type"] == "Property":
            compacted_value = value["value"]
            if "observedAt" in value:
                compacted_value["observedAt"] = value["observedAt"]
            compacted_document[key] = compacted_value
    
    return compacted_document

In [14]:
import json
e.pprint()
print(json.dumps(compact_ngsi_ld_document(json.loads(e.to_json())), indent=2))

{
  "id": "urn:ngsi-ld:Day:Cow007",
  "type": "Day",
  "@context": [
    "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"
  ],
  "date": {
    "type": "Property",
    "value": "2023-01-01"
  },
  "obs": {
    "type": "Property",
    "value": {
      "methaneyield": 210.0,
      "net_energy_requirement": 600.0
    },
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "pred": {
    "type": "Property",
    "value": {
      "methaneyield": 210.0,
      "net_energy_requirement": 600.0
    },
    "observedAt": "2023-01-01T00:00:00Z"
  }
}

{
  "id": "urn:ngsi-ld:Day:Cow007",
  "type": "Day",
  "@context": [
    "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"
  ],
  "date": "2023-01-01",
  "obs": {
    "methaneyield": 210.0,
    "net_energy_requirement": 600.0,
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "pred": {
    "methaneyield": 210.0,
    "net_energy_requirement": 600.0,
    "observedAt": "2023-01-01T00:00:00Z"
  }
}


### Using SmartDataModels code

In [15]:
import examples_conversor
norm = examples_conversor.normalized2keyvalues(json.loads(e.to_json()))
norm["observedAt"] = e["obs"]["observedAt"] # Conversion code drops observedAt fields
Day(**norm)

Day(type='Day', id='urn:ngsi-ld:Day:Cow007', date='2023-01-01', observedAt=datetime.datetime(2023, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), obs=AnimalDay(methaneyield=210.0, net_energy_requirement=600.0), pred=AnimalDay(methaneyield=210.0, net_energy_requirement=600.0))

### Schema

In [16]:
print(yaml.dump(Day.schema()))

definitions:
  AnimalDay:
    properties:
      methaneyield:
        title: Methaneyield
        type: number
      net_energy_requirement:
        title: Net Energy Requirement
        type: number
    required:
    - methaneyield
    - net_energy_requirement
    title: AnimalDay
    type: object
properties:
  date:
    title: Date
    type: string
  id:
    title: Id
    type: string
  obs:
    $ref: '#/definitions/AnimalDay'
  observedAt:
    format: date-time
    title: Observedat
    type: string
  pred:
    $ref: '#/definitions/AnimalDay'
  type:
    default: Day
    title: Type
    type: string
required:
- id
- date
- observedAt
- obs
- pred
title: Day
type: object

